trial

In [1]:
from datasets import load_dataset
from random import randrange

# Load dataset from the hub
dataset = load_dataset("medalpaca/medical_meadow_medqa", split="train")

print(f"Dataset Size: {len(dataset)}")
print(dataset[randrange(len(dataset))])
# Dataset Size: 10178

/Users/yy8/miniconda/envs/medAlpaca/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset Size: 10178
{'input': "Q:A 3-year-old boy is brought to the emergency department for nausea and vomiting for 1 day. His maternal uncle had a seizure disorder and died in childhood. He appears fatigued. Respirations are 32/min. Examination shows diffuse weakness in the extremities. Serum studies show a low pH, elevated lactate concentration, and normal blood glucose. A metabolic condition characterized by a defect in oxidative phosphorylation is suspected. Microscopic examination of a muscle biopsy specimen of this patient is most likely to show which of the following findings?? \n{'A': 'Fibrofatty replacement of normal muscle fibers', 'B': 'Muscle atrophy with perimysial inflammation', 'C': 'Intermyofibrillar accumulation of glycogen', 'D': 'Endomysial inflammation with T cell infiltration', 'E': 'Subsarcolemmal accumulation of mitochondria'},", 'instruction': 'Please answer with one of the option in the bracket', 'output': 'E: Subsarcolemmal accumulation of mitochondria'}


In [2]:
def format_prompt(sample):
    return f"""
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{sample["instruction"]}

### Input:
{sample["input"]}

### Response:
{sample["output"]}
"""

In [3]:
from random import randrange

print(format_prompt(dataset[randrange(len(dataset))]))


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Please answer with one of the option in the bracket

### Input:
Q:A 3-year-old boy is brought to the emergency department with abdominal pain. His father tells the attending physician that his son has been experiencing severe stomach aches over the past week. They are intermittent in nature, but whenever they occur he cries and draws up his knees to his chest. This usually provides some relief. The parents have also observed mucousy stools and occasional bloody stools that are bright red with blood clots. They tell the physician that their child has never experienced this type of abdominal pain up to the present. The boy was born at 39 weeks gestation via spontaneous vaginal delivery. He is up to date on all vaccines and is meeting all developmental milestones. On physical exam, his vitals are generally normal w

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Hugging Face model name
model_name = "llSourcell/medllama2_7b"
#"meta-llama/Llama-2-7b-chat-hf"
use_flash_attention = False

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [6]:
# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    use_cache=False,
    use_flash_attention_2=use_flash_attention,
    device_map="auto",
    torch_dtype=torch.float16
)

model.config.pretraining_tp = 1

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

TypeError: __init__() got an unexpected keyword argument 'use_flash_attention_2'

In [ ]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

# LoRA config based on QLoRA paper
peft_config = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.1,
    r=16,
    bias="none",
    task_type="CAUSAL_LM",
)
# Prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="finetuned-llama-7b-chat-hf-med",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    fp16=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    disable_tqdm=False
)

In [ ]:
from trl import SFTTrainer

max_seq_length = 1024 # max sequence length for model and packing of the dataset

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=format_prompt,
    args=args,
)